In [1]:

import pandas as pd
import regex as re
import nltk
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('93mwordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Error loading 93mwordnet: Package '93mwordnet' not found
[nltk_data]     in index
[nltk_data] Downloading package stopwords to /Users/aitor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aitor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# cargar excel con tweets
ruta = '/Users/aitor/Library/CloudStorage/OneDrive-UNIR/ASIGNATURAS/TFG/datasets/checkMix.xlsx'
df = pd.read_excel(ruta)

# convertir a lista
tweets = df.values.tolist()

df = pd.DataFrame(tweets, columns=['Fecha', 'Usuario', 'Displayed name', 'Contenido', 'label'])

print(df.head())
# Limpiar los tweets

# Eliminar filas con la columna 'Contenido' vacía
df.dropna(subset=['Contenido'], inplace=True)

# Eliminar filas duplicadas en todo el dataset
df.drop_duplicates(inplace=True)

# Restablecer el índice después de eliminar filas
df.reset_index(drop=True, inplace=True)

df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'http\S+', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'@(\w+)', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'#', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'\d+', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', str(x)))
df['Contenido'] = df['Contenido'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# remove stopwords and lemmatize tokens using apply() method
df['Contenido'] = df['Contenido'].apply(lambda x: x.lower())
df['Contenido'] = df['Contenido'].apply(lambda x: word_tokenize(x))
df['Contenido'] = df['Contenido'].apply(lambda x: [word for word in x if word not in stop_words])
df['Contenido'] = df['Contenido'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
df['Contenido'] = df['Contenido'].apply(lambda x: ' '.join(x))

# remove timezone information using tz_localize() method
df['Fecha'] = pd.to_datetime(df['Fecha']).dt.tz_localize(None)
## Guardar los tweets limpios en un archivo Excel ##
# ruta guardado
# y guardar los tweets en un archivo Excel
filename = "checkMixClean.xlsx"
ruta_carpeta = 'resultados/terrorismo/'
ruta_completa = ruta_carpeta + filename
# transfiero el dataframe a un archivo excel
df.to_excel(ruta_completa, index=False)


                Fecha         Usuario      Displayed name  \
0 2015-11-14 23:59:59  alysoncummings     Alyson Cummings   
1 2015-11-14 23:59:56      feedingjoy  Joy is a Lifestyle   
2 2015-11-14 23:59:55  StreamReporter     Stream Reporter   
3 2015-11-14 23:59:53  StreamReporter     Stream Reporter   
4 2015-11-14 23:59:48        Gagateux   AVEC LILY DEAR 🇫🇷   

                                           Contenido label  
0  pianist play john lennon imagine outside batac...   NaN  
1  happiness someone cycled bataclan paris play i...   NaN  
2  periscope rosscaneva rt mitney rosscaneva peri...   NaN  
3  periscope rosscaneva rt jmoon rosscaneva peris...   NaN  
4  mathias amp marie died bataclan yesterday didn...   NaN  
